In [395]:
import sys
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import sqlite3
import numpy as np
######################             Avocado prices                   #############################
#Sin nombre: 0
#Fecha
#Precio promedio
#Volumen total
#4046
#4225
#4770
#Total Bolsas
#Bolsas pequeñas
#Bolsas Grandes
#Bolsos XLarge
#tipo
#año
#región

#Date - La fecha de la observación.
#AveragePrice - El precio medio de un solo aguacate.
#type - convencional u orgánico
#year - el año
#Region - La ciudad o región de la observación.
#Total Volume - Número total de aguacates vendidos.
#4046 - Número total de aguacates con PLU 4046 vendidos
#4225 - Número total de aguacates con PLU 4225 vendidos
#4770 - Número total de aguacates con PLU 4770 vendidos

#Cuando compre el aguacate, puede verificar la madurez aplicándolo suavemente con la mano. Debe ser firme pero no duro. Cuando esté maduro, debe sentir una ligera cura cuando lo presiona con el pulgar. Lamentablemente verás en nuestro video que nunca podrás estar al 100% de lo que hay dentro. Esto no es diferente a cualquier otra fruta o verdura, pero sigue siendo decepcionante.

#Los aguacates normalmente se pegan con un número de PLU que te ayudará a identificar de qué tipo y de dónde son. Aquí hay una breve lista de los PLU de identificación.
#tipo de aguacate Hass
#4046 - Hass - pequeño
#4225 - Hass - grande
#4770 - Hass Extra Large
#4224 - Verde - Grande desde el oeste del río Mississippi
#4222 - Verde - Pequeño del oeste del río Mississippi
#4221 - Verde - Pequeño del este del río Mississippi
#4223 - Verde - Grande desde el este del río Mississippi
#4771 - Verde - Medio del este del río Mississippi
#3080 - variedad Pinkerton - todos los tamaños
#ORGÁNICO: cualquier PLU con un “9” que lo preceda es orgánico"""


In [396]:
data = pd.read_csv('avocado-prices/avocado.csv', sep=',')
#data['id'] = range(len(data))
data.columns

Index(['Unnamed: 0', 'Date', 'AveragePrice', 'Total Volume', '4046', '4225',
       '4770', 'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags', 'type',
       'year', 'region'],
      dtype='object')

In [397]:
#Note que la tabla con nombre Unnamed: 0 me da la semana del año en la cual se tomaron los datos por lo que
#decidi renombrar la coumna como week_year

In [398]:
data = data.rename(columns={'Unnamed: 0':'week_year'})
data.head()

,week_year,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [399]:
#1 Correccion de tipo de datos
for i in data.columns:
        print(i ,data[i].dtype)

data['Date'] = data['Date'].astype('datetime64')#lo cambie ya que corresponde a una fecha
print('Date' ,data['Date'].dtype, data['Date'][0])

week_year int64
Date object
AveragePrice float64
Total Volume float64
4046 float64
4225 float64
4770 float64
Total Bags float64
Small Bags float64
Large Bags float64
XLarge Bags float64
type object
year int64
region object
Date datetime64[ns] 2015-12-27 00:00:00


In [400]:
#2 VALORES FALTANTES
null_cols = data.isnull().sum()
#null_cols[null_cols > 0]
null_cols
#Al no tener ningun valor nulo decidi analizar la tabla

week_year       0
Date            0
AveragePrice    0
Total Volume    0
4046            0
4225            0
4770            0
Total Bags      0
Small Bags      0
Large Bags      0
XLarge Bags     0
type            0
year            0
region          0
dtype: int64

In [401]:
#3 VALORES INCORRECTOS

#Note que la coumna de XLarge Bags tenia muchos valores cero, y obtuve la siguiente relacion: 
#Total Bags = Small Bags + Large Bags + XLarge Bags

In [402]:
e=0
for i,v in data.iterrows():
    if data['XLarge Bags'][i] == 0:
        if 0 != round(data['Total Bags'][i]-data['Small Bags'][i] -data['Large Bags'][i],2):
            e=+1
print('El numero de errores es ', e)

El numero de errores es  1


In [403]:
#reasigne los valores correctos que estaban en cero

In [404]:
data.loc[(round(data['XLarge Bags'],2)==0), 'XLarge Bags'] = round(data['Total Bags']-data['Small Bags'] -data['Large Bags'],2)
data

,week_year,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.00,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,-0.00,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.00,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.00,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,-0.00,conventional,2015,Albany
5,5,2015-11-22,1.26,55979.78,1184.27,48067.99,43.61,6683.91,6556.47,127.44,-0.00,conventional,2015,Albany
6,6,2015-11-15,0.99,83453.76,1368.92,73672.72,93.26,8318.86,8196.81,122.05,0.00,conventional,2015,Albany
7,7,2015-11-08,0.98,109428.33,703.75,101815.36,80.00,6829.22,6266.85,562.37,-0.00,conventional,2015,Albany
8,8,2015-11-01,1.02,99811.42,1022.15,87315.57,85.34,11388.36,11104.53,283.83,-0.00,conventional,2015,Albany
9,9,2015-10-25,1.07,74338.76,842.40,64757.44,113.00,8625.92,8061.47,564.45,-0.00,conventional,2015,Albany


In [405]:
#4 Limpieza de texto y eliminación de caracteres especiales

In [406]:
print(set(data['week_year']))
print(set(data['Date']))
print(set(data['type']))
print(set(data['year']))
#todas las instancias estan bien definidas

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52}
{Timestamp('2017-10-15 00:00:00'), Timestamp('2015-02-22 00:00:00'), Timestamp('2017-04-16 00:00:00'), Timestamp('2015-09-27 00:00:00'), Timestamp('2017-01-01 00:00:00'), Timestamp('2017-11-26 00:00:00'), Timestamp('2016-09-25 00:00:00'), Timestamp('2015-08-30 00:00:00'), Timestamp('2017-11-12 00:00:00'), Timestamp('2016-04-03 00:00:00'), Timestamp('2017-12-10 00:00:00'), Timestamp('2015-05-24 00:00:00'), Timestamp('2016-10-23 00:00:00'), Timestamp('2016-08-21 00:00:00'), Timestamp('2016-11-06 00:00:00'), Timestamp('2016-09-18 00:00:00'), Timestamp('2017-06-11 00:00:00'), Timestamp('2016-04-17 00:00:00'), Timestamp('2017-09-24 00:00:00'), Timestamp('2015-12-13 00:00:00'), Timestamp('2015-01-04 00:00:00'), Timestamp('2015-02-15 00:00:00'), Timestamp('2017-02-26 00:00:00'), Timestamp('201

In [407]:
#5 Columnas de baja varianza

In [408]:
low_variance = []

for col in data._get_numeric_data():
    minimum = min(data[col])
    ninety_perc = np.percentile(data[col], 90)
    if ninety_perc == minimum:
        low_variance.append(col)

print(low_variance)
#No se genera ninguna columna que pueda eliminar

[]


In [409]:
#6 Encontrar y eliminar duplicados
before = len(data)
data = data.drop_duplicates()
after = len(data)
print('Number of duplicate records dropped: ', str(before - after))

Number of duplicate records dropped:  0


In [410]:
#7 Estadistica

In [411]:
stats = data.describe().transpose()
stats['IQR'] = stats['75%'] - stats['25%']
stats

,count,mean,std,min,25%,50%,75%,max,IQR
week_year,18249.0,24.232232,1.548104e+01,0.00,10.00,24.00,38.00,52.00,28.00
AveragePrice,18249.0,1.405978,4.026766e-01,0.44,1.10,1.37,1.66,3.25,0.56
Total Volume,18249.0,850644.013009,3.453545e+06,84.56,10838.58,107376.76,432962.29,62505646.52,422123.71
4046,18249.0,293008.424531,1.264989e+06,0.00,854.07,8645.30,111020.20,22743616.17,110166.13
4225,18249.0,295154.568356,1.204120e+06,0.00,3008.78,29061.02,150206.86,20470572.61,147198.08
4770,18249.0,22839.735993,1.074641e+05,0.00,0.00,184.99,6243.42,2546439.11,6243.42
Total Bags,18249.0,239639.202060,9.862424e+05,0.00,5088.64,39743.83,110783.37,19373134.37,105694.73
Small Bags,18249.0,182194.686696,7.461785e+05,0.00,2849.42,26362.82,83337.67,13384586.80,80488.25
Large Bags,18249.0,54338.088145,2.439660e+05,0.00,127.47,2647.71,22029.25,5719096.61,21901.78
XLarge Bags,18249.0,3106.426781,1.769289e+04,-1.00,0.00,0.00,132.50,551693.65,132.50


In [414]:
#De los datos apreciamos que solo podemos tomar las siguientes columnas
outliers = pd.DataFrame(columns=data.columns)

for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = data[(data[col] >= lower) | #cambie la desigualdad ya que solo me interesan los datos limpios
                   (data[col] <= upper)].copy()#cambie la desigualdad ya que solo me interesan los datos limpios
    results['Outlier'] = col
    outliers = outliers.append(results)
outliers=outliers[['week_year', 'AveragePrice', 'Total Volume', '4046', '4225',
       '4770', 'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags', 'type']]
outliers

,week_year,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type
0,0,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.00,conventional
1,1,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,-0.00,conventional
2,2,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.00,conventional
3,3,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.00,conventional
4,4,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,-0.00,conventional
5,5,1.26,55979.78,1184.27,48067.99,43.61,6683.91,6556.47,127.44,-0.00,conventional
6,6,0.99,83453.76,1368.92,73672.72,93.26,8318.86,8196.81,122.05,0.00,conventional
7,7,0.98,109428.33,703.75,101815.36,80.00,6829.22,6266.85,562.37,-0.00,conventional
8,8,1.02,99811.42,1022.15,87315.57,85.34,11388.36,11104.53,283.83,-0.00,conventional
9,9,1.07,74338.76,842.40,64757.44,113.00,8625.92,8061.47,564.45,-0.00,conventional


In [415]:
outliers.to_csv('cleaned.csv')